In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from CA1_Yolo import *
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
img_dir = "../data/image/"
anno_dir = "../data/annotation/"

In [3]:
all_anno, labels = read_annotation_files(img_dir, anno_dir)

In [4]:
anchor_boxes = generateAnchorBoxes(all_anno, labels)

iter: 1 distances: 704.1744880550395
iter: 2 distances: 62.29262774623318
iter: 3 distances: 31.093210949668126
iter: 4 distances: 39.79917129364537
iter: 5 distances: 26.149889309666506
iter: 6 distances: 17.95296032157585
iter: 7 distances: 9.501443132099507
iter: 8 distances: 2.1203066680569482
iter: 9 distances: 5.692698625187262
iter: 10 distances: 2.1160483495387536


In [5]:
anchor_boxes

[30, 19, 35, 25, 44, 28, 51, 38, 56, 28, 60, 52, 63, 38, 72, 47, 93, 93]

In [6]:
train_size=0.8
train_valid_split = int(0.8 * len(all_anno))
np.random.shuffle(all_anno)
train_anno = all_anno[:train_valid_split]
valid_anno = all_anno[train_valid_split:]
max_boxes = max([len(anno['object']) for anno in (train_anno + valid_anno)])

In [7]:
len(train_anno)

63

In [8]:
train_generator = DataGenerator(
    annotations=train_anno,
    max_boxes=max_boxes,
    anchors=anchor_boxes,
    labels=labels,
    batch_size=16,
    width=416,
    height=416,
    shuffle=True,
    augment=True)

In [9]:
valid_generator = DataGenerator(
    annotations=valid_anno,
    max_boxes=max_boxes,
    anchors=anchor_boxes,
    labels=labels,
    batch_size=16,
    width=416,
    height=416,
    shuffle=True,
    augment=True)

In [10]:
train_model, infer_mode = YoloV3(
    numcls=len(labels),
    anchors=anchor_boxes,
    max_grid=[416, 416],
    batch_size=16,
    threshold=0.5,
    max_boxes=max_boxes)







Instructions for updating:
Use `tf.cast` instead.



In [11]:
train_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
zero_paddi

In [12]:
def dummy_loss(y_true, y_pred):
    return tf.sqrt(tf.reduce_sum(y_pred))

In [13]:
opt = Adam(lr=1e-4, clipnorm=0.001)
train_model.compile(loss=dummy_loss, optimizer=opt)

In [14]:
callback = create_callbacks()

D:\Anaconda3\envs\rtav\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [15]:
train_model.fit_generator(
    generator=train_generator,
    steps_per_epoch=len(train_generator) * 3,
    epochs=300,
    callbacks=callback
)

Epoch 1/300
Curr ind:16, next_indices:32
Curr ind:32, next_indices:48
resizing:  416 416
Curr ind:0, next_indices:16


ValueError: negative dimensions are not allowed